<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Загрузим библиотеки

In [1]:
import pandas as pd
!pip3 install pandas-profiling==2.11
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 243 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 679 kB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 58.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 44.2 MB/s eta 0:00:01
     |████████████████████████████████| 296 kB 41.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 46.2 MB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27085 sha256=c3f083d2a633a9f7c13ebf844633856ae585a4eaddec4f546ba403308a16e6ba
  Stored in directory: /home/jovyan/.cache/pip/wheels/1d/05/04/c6d7d3b66539d9e659ac6dfe81e2d0fd4c1a8316cc5a403300
Successfully built htmlmin


In [2]:
try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
profile = ProfileReport(data,
                        title='Pandas Profiling Report - Data 0')

profile.to_widgets()

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Вывод: пропусков нет, есть дубликаты

In [6]:
data = data.drop_duplicates()

In [7]:
data.duplicated().sum()

0

Исправим названия столбцов

In [8]:
data.columns

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')

In [9]:
data.columns = ['gender', 'age', 'income', 'family_members', 'insurance_payments']

In [10]:
data.columns

Index(['gender', 'age', 'income', 'family_members', 'insurance_payments'], dtype='object')

Вывод: открыли файл, посмотрели информацию о данных, посмотрели сами данные, с помощью profiling посмотрели на графики распределения данных, посмотрели, есть ли дубликаты и пропуски, посмотрели, есть ли взаимосвязи, есть связь между возрастом и выплатами, удалили дубликаты, так как это повторяющиеся строки, возможно из-за технологической ошибки, при выгрузке или загрузке данных образовалось дублирование, поправили названия столбцов на английский и на snake_case.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Преобразование в формулах:

$a'=X'w'=(XP)w'$

$a'= XP((XP)^TXP)^{-1}(XP)^Ty$
    
$a' = XP (P^TX^TXP)^{-1}P^TX^Ty$
    
$a' = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty$
    
где $ PP^{-1} = E$ и $(P^T)^{-1}P^T = E$

$a'= XE(X^TX)^{-1}EX^Ty$
    
$a' = X(X^TX)^{-1}X^Ty = a$
 

## Алгоритм преобразования

**Алгоритм**

Проведем обучение и тестирование на изначальных данных. Возьмем изначальные данные, преобразуем их в матрицу, умножим их на случайную обратимую матрицу, затем проведем обучение и тестирование.

**Обоснование**

сравним результаты тестирования и метрики R2 и сделаем вывод о том, что предсказания совпадают в обоих вариантах, если данные совпадут

## Проверка алгоритма

Обучим модель на изначальных данных

In [11]:
features = data.drop('insurance_payments', axis=1).values
target = data['insurance_payments'].values

Разделим выборку на обучающую и тестовую

In [12]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345) 


создадим класс для нашей модели линейной регресии

In [13]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X)).dot(X.T)).dot(y)# < напишите код здесь >
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Изначальные данные: обучающие и тестовые данные

In [14]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions_train = model.predict(features_train)
print(r2_score(target_train, predictions_train))

0.43334816364347406


In [15]:
model = LinearRegression()
model.fit(features_test, target_test)
predictions_test = model.predict(features_test)
print(r2_score(target_test, predictions_test))

0.4195675218104399


создадим матрицу для изменения данных

In [16]:
matrix = np.random.rand(4,4)
matrix

array([[0.20520109, 0.44693163, 0.93399887, 0.4413386 ],
       [0.69761454, 0.28698227, 0.82903695, 0.43443635],
       [0.55110537, 0.64673288, 0.28466691, 0.43746348],
       [0.98959359, 0.42881704, 0.88617302, 0.67716573]])

Изменим данные

In [17]:
features_train_changed = features_train @ matrix

Измененные обучающие данные

In [18]:
model = LinearRegression()
model.fit(features_train_changed, target_train)
predictions_train_changed = model.predict(features_train_changed)
print(r2_score(target_train, predictions_train_changed))

0.4333481636418448


Изменим тестовые данные и проверим модель на тестовых данных

In [19]:
features_test_changed = features_test @ matrix

In [20]:
model = LinearRegression()
model.fit(features_test_changed, target_test)
predictions_test_changed = model.predict(features_test_changed)
print(r2_score(target_test, predictions_test_changed))

0.419567521809575


Вывод: проверили модель на изначальных и преобразованных данных, получили одинаковый результат, значит изменение данных не повлияло на модель.